# Visualising Incident Data on Interactive Maps with Folium

### Introduction

Interactive maps are powerful tools for visualising geographical data. In this blog post, we will explore how to use the [Folium](https://github.com/python-visualization/folium) library in Python to create an interactive map.

The dataset we'll be using is part of a project I am working on to analyze incident data collected by the Loch Lomond Rescue Boat ([LLRB](https://www.lochlomondrescueboat.org/)). The LLRB is a volunteer-run organization that provides emergency response and rescue services on Loch Lomond in Scotland.

This project aims to optimize their operations and enhance public safety by analyzing three key areas: **preparedness**, **prediction**, and **public awareness**. To find out more about the project, visit the [here](https://github.com/rhyslwells/LLRB). If you would like to contribute, please get in touch.

The dataset was collected between 2021-2023 and includes details such as the date of the incident, crew members involved, and specific details of the incident.

#### Importing Libraries and Data

Firstly, we need to import the necessary libraries and read the data into a pandas DataFrame.


In [1]:
import folium
from folium.plugins import MarkerCluster
import pandas as pd

df = pd.read_csv("..\..\data\cleaned.csv")
df[['latitude', 'longitude']] = df['core_lat_long'].str.split(',', expand=True)
df['latitude'] = pd.to_numeric(df['latitude'], errors='coerce')
df['longitude'] = pd.to_numeric(df['longitude'], errors='coerce')
feats=["latitude", "longitude"]
df = df.dropna(subset=feats)
feats=[ "latitude", "longitude","date_of_shout","pager_code","shout_details_tags","crew_on_board","crew_on_shore","shout_details"]
df_filtered = df[feats]
df_filtered.head(2)

,latitude,longitude,date_of_shout,pager_code,shout_details_tags,crew_on_board,crew_on_shore,shout_details
0,56.090115,-4.617764,21/07/2023,999,"FalseAlarm, Miscellaneous","RB, TR, AM,CS","GH, DON","Call on channel 16 saying swimmer in water, in..."
1,56.302002,-4.720782,19/07/2023,333,"Mechanical, Rescue, Transport","RB, CG, TR, ABJ, AM","RO, ABS",17' Fletcher speedboat with 1 male occupant ha...


#### Initialising the Map

We'll create a base map centered at a specific latitude and longitude.


In [3]:
map_center = df_filtered[['latitude', 'longitude']].dropna().mean().tolist()
# Create the map object
m = folium.Map(location=map_center,control_scale=True,zoom_control=True,zoom_start=10.45)

#### Adding Markers


Next we map the location of where the boat is stored.

In [4]:
LLRB_pos = [56.10063229251386, -4.635696317637836]
# Add the LLRB marker
folium.Marker(
    location=LLRB_pos,
    tooltip="LLRB position",
    icon=folium.Icon(color="blue"),
).add_to(m)

To manage a large number of markers efficiently, we use Marker Clusters.

In [5]:
marker_cluster = MarkerCluster().add_to(m)

We then mark the incident locations. We do so by iterating through each row of the DataFrame and adding markers for each incident to the map with popups and tooltips that provide additional information about each incident. We color-code the markers based on the type of incident. Green for minor incidents, orange for search and rescue, and red for major incidents.

In [6]:
# Define color mapping for pager codes
color_mapping = {222: "green", 333: "orange", 999: "red"}

# Iterate through each row of df_filtered
for index, row in df_filtered.iterrows():
    lat = row["latitude"]
    lng = row["longitude"]
    descrip = f"{row['date_of_shout']}\nCrew on board: {row['crew_on_board']}\nCrew on shore: {row['crew_on_shore']}\nShout tags: {row['shout_details_tags']}"
    details = f"Shout details:\n{row['shout_details']}"
    
    # Add incident marker to the marker cluster
    folium.Marker(
        location=[lat, lng],
        tooltip=descrip,
        popup=folium.Popup(details, max_width=300),
        icon=folium.Icon(color=color_mapping[row['pager_code']]),
    ).add_to(marker_cluster)

#### The map

Finally, we return and display the map.


In [7]:
m

### Conclusion

With just a few lines of code, Folium allows you to create interactive maps that provide valuable insights into geographical data. We have already used this tool to help stakeholders analyse historical data, as seen in our [report](https://github.com/rhyslwells/LLRB/blob/main/docs/report.docx). We foresee integrating such maps into the LLRB website to provide the public with real-time information on incidents in an accessible manner.

Our project with the Loch Lomond Rescue Boat (LLRB) explores various avenues to enhance operational efficiency and public safety using these tools. Our next project will involve using social media data to gauge public awareness of the LLRB and identify areas for improvement.

For more details, visit [our GitHub repository](https://github.com/rhyslwells/LLRB). If you have any questions or want to contribute, please get in touch. Thank you for following along, see you in the next post!